In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage.interpolation import shift

F:\ML\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
height = 28
width = 28
channels = 1
n_inputs = height * width

In [3]:
conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

In [4]:
conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

In [5]:
pool3_fmaps = conv2_fmaps

In [6]:
n_fc1 = 64
n_outputs = 1

In [7]:
tf.reset_default_graph()

In [8]:
with tf.name_scope("inputs"):
    X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize, 
                             strides=conv1_stride, padding=conv1_pad, 
                             activation=tf.nn.relu, name="conv1")
    conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize, 
                             strides=conv2_stride, padding=conv2_pad, 
                             activation=tf.nn.relu, name="conv2")

In [9]:
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

In [10]:
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

In [11]:
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [12]:
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

In [13]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [14]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [15]:
MNIST_PATH = os.path.join("datasets", "MNIST")
FILE_NAME = "train.csv"

In [16]:
def load_mnist_data(mnist_path=MNIST_PATH, file_name = FILE_NAME):
    csv_path = os.path.join(mnist_path, file_name)
    return pd.read_csv(csv_path)

In [17]:
def random_batch(X_train, y_train, batch_size):
    rnd_index = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_index, :]
    y_batch = y_train[rnd_index]
    return np.array(X_batch), np.array(y_batch)

In [18]:
mnist = load_mnist_data()

In [19]:
X_train, X_val, y_train, y_val = train_test_split(mnist.values[:, 1:], mnist.values[:, 0], test_size = 0.2, random_state = 42)

In [ ]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(X_train.shape[0]/ batch_size))

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(n_batches):
            X_batch, y_batch = random_batch(X_train, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print(epoch, "Точность при обучении:", acc_train,
                     "Точность при проверке:", acc_test)